# Generation of WB SVG Map

1. Download data from: [https://datacatalog.worldbank.org/dataset/world-subnational-boundaries](https://datacatalog.worldbank.org/dataset/world-subnational-boundaries) → [https://ddh-s3-official-internal.worldbank.org/ddhfiles/93822/geospatial/Subnational_Boundaries/country_polygons.zip](https://ddh-s3-official-internal.worldbank.org/ddhfiles/93822/geospatial/Subnational_Boundaries/country_polygons.zip)
2. Go to [https://mapshaper.org/](https://mapshaper.org/) and upload the shapefile.
3. Save as SVG and CSV.
4. Process SVG file to include data from CSV. The order of the path appears to be in the same order as the rows in the CSV.
5. Take note of the "Rules" column.

In [1]:
import pandas as pd
from wb_nlp import dir_manager
import json

import xml
import xml.etree.ElementTree as ET

ET.register_namespace('', "http://www.w3.org/2000/svg")

import re
SPECIAL_NAME_PATTERN = re.compile(r"(\S+), (.+)")

def process_special_name(value):
    value = SPECIAL_NAME_PATTERN.sub(r"\2 \1", value)
    return value

In [2]:
with open(dir_manager.get_path_from_root("app", "app_kcp", "public", "static", "data", "iso3166-3-country-info.json")) as json_file:
    iso3map = json.load(json_file)

meta = pd.read_csv(dir_manager.get_data_dir("maps", "WB", "WB_CountryPolys.csv"))
set(meta["ISO_Codes"].values).difference(iso3map)

{'KOS', 'XXX'}

In [3]:
meta["title"] = meta["Names"]
special_name_filter = meta["Names"].str.contains(',') & meta["Rules"].str.contains("Always have .* before")

meta.loc[special_name_filter, "title"] = meta[special_name_filter]["title"].map(process_special_name)

meta["id"] = meta["ISO_Codes"]
meta.loc[meta["id"] == "ESH", "id"] = "XXX"
meta.loc[meta["id"] == "XXX", "id"] = meta.loc[meta["id"] == "XXX", "id"] + "_" + meta.loc[meta["id"] == "XXX", "title"].str.lower().str.replace(" ", "_")

In [4]:
import copy

In [5]:
simplified = True

if simplified:
    tree = ET.parse(dir_manager.get_data_dir("maps", "WB", "WB_CountryPolys - Simplified.svg"))
else:
    tree = ET.parse(dir_manager.get_data_dir("maps", "WB", "WB_CountryPolys.svg"))

root = tree.getroot()
root.attrib["id"] = "map-svg"
remove_p = []

for child in root:
    # child=g
    xxx_p = []
    for m, p in zip(meta.to_dict("records"), child):
        id = iso3map.get(m["id"], {}).get("alpha-2", m["id"])
        if simplified and id == "TF":
            # French Southern and Antarctic Lands (Fr.)
            remove_p.append(p)
            continue

        p.attrib["id"] = iso3map.get(m["id"], {}).get("alpha-2", m["id"])
        p.attrib["title"] = m["title"]
        p.attrib["class"] = "land"
        p.attrib["d"] = p.attrib["d"]

        if m["id"].startswith("XXX_"):
            pp = copy.deepcopy(p)
            p.attrib["id"] = p.attrib["id"] + "_mask"
            p.attrib["class"] = f'{p.attrib["class"]} xxx-mask'
            pp.attrib["class"] = f'{pp.attrib["class"]} xxx'

            xxx_p.append(pp)

    for pp in xxx_p:
        child.append(pp)

for p in remove_p:
    child.remove(p)

In [6]:
len(list(child))

259

In [7]:
# Use some attributes from original vue-map-chart Map.
root.attrib["width"] = "100%"
root.attrib.pop("height")

'320'

In [8]:
tree.write(dir_manager.get_data_dir("maps", "WB", "WB_CountryPolys-00-avs-processes-wb-map-svg.svg"))

In [10]:
# width="100%" viewBox="0 0 1008 650"

In [12]:
1008 / 650

1.5507692307692307

In [13]:
800 / 320

2.5